In [40]:

# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import jupyter_utils
importlib.reload(jupyter_utils)

ModuleNotFoundError: No module named 'jupyter_utils'

# PA4 Simple Classifiers
## Part 2: Auto Dataset Classification  
Write a Jupyter Notebook (pa4.ipynb) that uses your mysklearn package to build simple classifiers for the "pre-processed" automobile dataset (auto-data-removed-NA.txt) you created for PA2. In the Notebook, describe the steps, log any assumptions and/or issues you had in doing the steps, and provide insights on the step results. All re-usable utility functions should be separate from your Notebook in an appropriate module.

### Step 1 Train/Test Sets: Random Instances and Linear Regression  
Create a classifier that predicts mpg values using (least squares) linear regression based on vehicle weight. Your classifier should take one or more instances, compute the predicted MPG values, and then map these to the DOE classification/ranking (given in PA3) for each corresponding instance. Test your classifier by selecting 5 random instances from the dataset, predict their corresponding mpg ranking, and then show their actual mpg ranking as follows:

* Changes:
    * I had an issue with using fit() on two different lists, one has nested list values and one was a list of values. I had to add to the fit() function to account for this issue. 



In [33]:
import os
import random

fname = os.path.join("input_data", "auto-data-removed-NA.txt")
auto_data = MyPyTable().load_from_file(fname)
auto_data.remove_rows_with_missing_values()

weight = auto_data.get_column('weight')
weight = [[val] for val in weight]
a, b, c, d = myevaluation.train_test_split(weight, auto_data.get_column('mpg'), shuffle=True)

myline = MySimpleLinearRegressor()
myline.fit(a,c)

rand_rows = jupyter_utils.get_rand_rows(auto_data, 5)
print('===========================================\n\
STEP 1: Linear Regression MPG Classifier\n\
===========================================')
for row in rand_rows:
    print('instance:', row)
    print('class:', jupyter_utils.get_rating(myline.predict([[row[4]]])[0]), 'actual: ', juputils.get_rating(row[0]))

fit
STEP 1: Linear Regression MPG Classifier
instance: [27.2, 4.0, 141.0, 71.0, 3190.0, 24.8, 79.0, 2.0, 'peugeot 504', 8040.0]
class: 5 actual:  7
instance: [15.0, 8.0, 350.0, 145.0, 4082.0, 13.0, 73.0, 1.0, 'chevrolet monte carlo s', 3562.0]
class: 3 actual:  3
instance: [15.5, 8.0, 350.0, 170.0, 4165.0, 11.4, 77.0, 1.0, 'chevrolet monte carlo landau', 4968.0]
class: 2 actual:  3
instance: [14.0, 8.0, 455.0, 225.0, 4425.0, 10.0, 70.0, 1.0, 'pontiac catalina', 3164.0]
class: 1 actual:  2
instance: [14.0, 8.0, 400.0, 175.0, 4464.0, 11.5, 71.0, 1.0, 'pontiac catalina brougham', 4154.0]
class: 1 actual:  2


### Step 2 Train/Test Sets: Random Instances and kNN  
Create a nearest neighbor classifier for mpg that uses the number of cylinders, weight, and acceleration attributes to predict mpg for k = 5. Be sure to normalize the three attribute values and also use the Euclidean distance metric. Similar to Step 1, test your classifier by selecting 5 random instances from the dataset, predict their corresponding mpg ranking, and then show their actual mpg ranking:



In [34]:
print('\n===========================================\n\
STEP 2: k=5 Nearest Neighbor MPG Classifier\n\
===========================================')
myk = MyKNeighborsClassifier(5)

weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, _ = myutils.scale(x_vals, [[]])

a, b, c, d = myevaluation.train_test_split(scaled_x, auto_data.get_column('mpg'), shuffle=True)

myk.fit(a,auto_data.get_column('mpg'))

rand_rows = jupyter_utils.get_rand_rows(auto_data, 5)
for row in rand_rows:
    print('instance:', row)
    _, scaled_test = myutils.scale(x_vals, [[row[4],row[1],row[5]]])
    print('class:', jupyter_utils.get_rating(myk.predict(scaled_test)[0]), 'actual: ', juputils.get_rating(row[0]))


STEP 2: k=5 Nearest Neighbor MPG Classifier
instance: [13.0, 8.0, 400.0, 170.0, 4746.0, 12.0, 71.0, 1.0, 'ford country squire (sw)', 4074.0]
class: 7 actual:  1
instance: [20.0, 4.0, 140.0, 90.0, 2408.0, 19.5, 72.0, 1.0, 'chevrolet vega', 2160.0]
class: 2 actual:  5
instance: [19.0, 6.0, 250.0, 100.0, 3282.0, 15.0, 71.0, 1.0, 'pontiac firebird', 3168.0]
class: 6 actual:  4
instance: [26.0, 4.0, 97.0, 78.0, 2300.0, 14.5, 74.0, 2.0, 'opel manta', 3275.0]
class: 2 actual:  6
instance: [14.0, 8.0, 350.0, 165.0, 4209.0, 12.0, 71.0, 1.0, 'chevrolet impala', 3813.0]
class: 2 actual:  2


### Step 3 Train/Test Sets: Random Sub-sampling
Compute the predictive accuracy and error rate of the two classifiers using random sub-sampling with k = 10. Your output should look something like this (where the ??'s should be replaced by actual values):



In [35]:
print('\n===========================================\n\
STEP 3: Predictive Accuracy\n\
===========================================\n\
Random Subsample (k=10, 2:1 Train/Test)')
auto_data = MyPyTable().load_from_file(fname)
auto_data.remove_rows_with_missing_values()

weight = auto_data.get_column('weight')
weight = [[val] for val in weight]
a, b, c, d = myevaluation.train_test_split(weight, auto_data.get_column('mpg'), shuffle=True)

myline = MySimpleLinearRegressor()
myline.fit(a,c)
predicted = myline.predict(b)
predicted = [jupyter_utils.get_rating(val) for val in predicted]
expected = [jupyter_utils.get_rating(val) for val in d]
correct = 0
for i in range(len(predicted)):
    if predicted[i] == expected[i]:
        correct += 1
print('Linear Regression: accuracy = ', correct/len(predicted), 'error rate =', 1-correct/len(predicted))

weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, _ = myutils.scale(x_vals, [[]])

a, b, c, d = myevaluation.train_test_split(scaled_x, auto_data.get_column('mpg'), shuffle=True)
myk = MyKNeighborsClassifier(10)
myk.fit(a,c)

predicted = myk.predict(b)
predicted = [jupyter_utils.get_rating(val) for val in predicted]
expected = [jupyter_utils.get_rating(val) for val in d]
correct = 0
for i in range(len(predicted)):
    if predicted[i] == expected[i]:
        correct += 1
print('K Nearest Neighbors: accuracy = ', correct/len(predicted), 'error rate =', 1-correct/len(predicted))


STEP 3: Predictive Accuracy
Random Subsample (k=10, 2:1 Train/Test)
fit
Linear Regression: accuracy =  0.37209302325581395 error rate = 0.627906976744186
K Nearest Neighbors: accuracy =  0.20930232558139536 error rate = 0.7906976744186046


### Step 4 Train/Test Sets: Cross Validation
Compute the predictive accuracy and error rate of the two classifiers using separate training and test sets. You should use mpg rankings for both k-fold cross validation and stratified k-fold cross validation with k = 10. Your output should look something like this (where the ??'s should be replaced by actual values):

In [38]:
importlib.reload(mysklearn.myclassifiers)
importlib.reload(mysklearn.myevaluation)
importlib.reload(mysklearn.myutils)

print('\n===========================================\n\
STEP 4: Predictive Accuracy\n\
===========================================\n\
10-Fold Cross Validation')
weight = [[val] for val in auto_data.get_column('weight')]
train_folds, test_folds = myevaluation.kfold_cross_validation(weight, 10)
X_train = []
y_train = []

y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]


X_train, y_train, X_test, y_test = myutils.get_from_folds(weight, y, train_folds, test_folds)

myline = MySimpleLinearRegressor()
myline.fit(X_train,y_train)

predicted_linear = [round(val) for val in myline.predict(X_test)]
actual_linear = y_test

count = jupyter_utils.count_correct(predicted_linear, y_test)
print('Linear Regression: accuracy =', count/len(predicted_linear), 'error rate =', 1 - count/len(predicted_linear))

weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, _ = myutils.scale(x_vals, [[]])

train_folds, test_folds = myevaluation.kfold_cross_validation(scaled_x, 10)
y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]

X_train, y_train, X_test, y_test = myutils.get_from_folds(scaled_x, y, train_folds, test_folds)

myk = MyKNeighborsClassifier(10)
myk.fit(X_train,y_train)

predicted_neighbors = myk.predict(X_test)
actual_neighbors = y_test

count = jupyter_utils.count_correct(predicted_neighbors, y_test)

print('K Nearest Neighbors: accuracy =', count/len(predicted_neighbors), 'error rate =', 1 - count/len(predicted_neighbors))

weight = [[val] for val in auto_data.get_column('weight')]
y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(weight, y, 10)
X_train = []
y_train = []

y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]


X_train, y_train, X_test, y_test = myutils.get_from_folds(weight, y, train_folds, test_folds)

myline = MySimpleLinearRegressor()
myline.fit(X_train,y_train)

predicted_linear_stratified = [round(val) for val in myline.predict(X_test)]
actual_linear_stratified = y_test

count = jupyter_utils.count_correct(predicted_linear_stratified, y_test)
print('\nStratified 10-Fold Cross Validation\nLinear Regression: accuracy =', count/len(predicted_linear_stratified), 'error rate =', 1 - count/len(predicted_linear_stratified))

weight = auto_data.get_column('weight')
cylinders = auto_data.get_column('cylinders')
acceleration = auto_data.get_column('acceleration')

x_vals = [[weight[i], cylinders[i],acceleration[i]] for i in range(len(weight))]
scaled_x, _ = myutils.scale(x_vals, [[]])

y = [jupyter_utils.get_rating(val) for val in auto_data.get_column('mpg')]
train_folds, test_folds = myevaluation.stratified_kfold_cross_validation(scaled_x, y, 10)

X_train, y_train, X_test, y_test = myutils.get_from_folds(scaled_x, y, train_folds, test_folds)

myk = MyKNeighborsClassifier(10)
myk.fit(X_train,y_train)

predicted_neighbors_stratified = myk.predict(X_test)
actual_neighbors_stratified = y_test

count = jupyter_utils.count_correct(predicted_neighbors_stratified, y_test)

print('K Nearest Neighbors: accuracy =', count/len(predicted_neighbors_stratified), 'error rate =', 1 - count/len(predicted_neighbors_stratified))


STEP 4: Predictive Accuracy
10-Fold Cross Validation
fit
Linear Regression: accuracy = 0.8185328185328186 error rate = 0.18146718146718144
K Nearest Neighbors: accuracy = 0.528957528957529 error rate = 0.471042471042471
fit

Stratified 10-Fold Cross Validation
Linear Regression: accuracy = 0.3166023166023166 error rate = 0.6833976833976834
K Nearest Neighbors: accuracy = 0.14671814671814673 error rate = 0.8532818532818532


### Step 5 Confusion Matrices  
Create confusion matrices for each classifier based on the stratified 10-fold cross validation results. You can use the tabulate package to display your confusion matrices (it is also okay to format the table manually). Here is an example:




In [39]:
importlib.reload(jupyter_utils)
print('\n===========================================\n\
STEP 5: Confusion Matrices\n\
===========================================\n')
matrix = myevaluation.confusion_matrix(actual_linear_stratified, predicted_linear_stratified, [0,1,2,3,4,5,6,7,8,9,10])
table_header = ['MPG', 1,2, 3 ,4, 5, 6, 7, 8, 9, 10, 'Total', 'Recognition (%)']
jupyter_utils.add_conf_stats(matrix)

print('Linear Regression (Stratified 10-fold Cross Validation Results)')
jupyter_utils.print_tabulate(matrix, table_header)

matrix = myevaluation.confusion_matrix(actual_neighbors_stratified, predicted_neighbors_stratified, [0,1,2,3,4,5,6,7,8,9,10])
jupyter_utils.add_conf_stats(matrix)

print('\nK Neighbors (Stratified 10-fold Cross Validation Results)')
jupyter_utils.print_tabulate(matrix, table_header)


STEP 5: Confusion Matrices

Linear Regression (Stratified 10-fold Cross Validation Results)
=====  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
  MPG    1    2    3    4    5    6    7    8    9    10    Total    Recognition (%)
=====  ===  ===  ===  ===  ===  ===  ===  ===  ===  ====  =======  =================
    1    8   10    8    0    1    0    0    0    0     0       28              28.57
    2    2    9    4    0    1    0    0    0    0     0       18              50
    3    1   14   12    8    1    0    0    0    0     0       39              30.77
    4    0    1   11   25   11    2    1    0    0     0       55              45.45
    5    0    0    1   12   19   15    0    0    0     0       52              36.54
    6    0    0    0    1    6   21    6    0    0     0       40              52.5
    7    0    0    0    1    0    8   18    0    0     0       34              52.94
    8    0    0    0    0    0    1   16    1    0     0     